In [1]:
# import h5py
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, Subset
import h5py
import numpy as np

import h5py

import h5py
import numpy as np

# Load test data from test_raw.h5
with h5py.File("./data/test_psd.h5", "r") as f_test:
    X_test = f_test['data'][:]
    Y_test = f_test['labels'][:]
    S_test = f_test['sessions'][:]
    #H_test = f_test['hardwares'][:]  # Load the 'hardwares' dataset as h_test


# Now, X_combined, Y_combined, S_combined, and h_combined contain the merged data


# Now, X_combined, Y_combined, and S_combined contain the merged data


# Shuffle the data
indices = np.arange(X_test.shape[0])
np.random.shuffle(indices)

# Reorder the arrays according to the shuffled indices
x_test = X_test[indices]
y_test = Y_test[indices]
s_test = S_test[indices]
#h_test = H_test[indices]


# Find unique subjects
unique_subjects = np.unique(y_test)

# Initialize lists to hold the data for x_test_e and x_test_v
x_test_e_list = []
y_test_e_list = []
s_test_e_list = []
h_test_e_list = []

x_test_v_list = []
y_test_v_list = []
s_test_v_list = []
h_test_v_list = []

# Assign the minimum session for each subject to x_test_e and the rest to x_test_v
for subject in unique_subjects:
    subject_indices = np.where(y_test == subject)[0]
    subject_sessions = s_test[subject_indices]
    
    # Skip subjects with fewer than two unique sessions
    if len(np.unique(subject_sessions)) < 2:
        continue

    print(f"Subject {subject}, Number of unique sessions: {len(np.unique(subject_sessions))}")
    
    # Assign the minimum session to the evaluation set (x_test_e)
    min_session = np.min(subject_sessions)
    
    # Append data to the evaluation set (min session)
    x_test_e_list.extend(x_test[subject_indices][subject_sessions == min_session])
    y_test_e_list.extend(y_test[subject_indices][subject_sessions == min_session])
    s_test_e_list.extend(s_test[subject_indices][subject_sessions == min_session])
    #h_test_e_list.extend(h_test[subject_indices][subject_sessions == min_session])

    # Append remaining sessions to the validation set (x_test_v)
    x_test_v_list.extend(x_test[subject_indices][subject_sessions != min_session])
    y_test_v_list.extend(y_test[subject_indices][subject_sessions != min_session])
    s_test_v_list.extend(s_test[subject_indices][subject_sessions != min_session])
    #h_test_v_list.extend(h_test[subject_indices][subject_sessions != min_session])

# Shuffle and convert lists back to numpy arrays for x_test_e
indices_e = np.arange(len(x_test_e_list))
np.random.shuffle(indices_e)

x_test_e = np.array(x_test_e_list)[indices_e]
y_test_e = np.array(y_test_e_list)[indices_e]
s_test_e = np.array(s_test_e_list)[indices_e]
#h_test_e = np.array(h_test_v_list)[indices_e]

# Shuffle and convert lists back to numpy arrays for x_test_v
indices_v = np.arange(len(x_test_v_list))
np.random.shuffle(indices_v)

x_test_v = np.array(x_test_v_list)[indices_v]
y_test_v = np.array(y_test_v_list)[indices_v]
s_test_v = np.array(s_test_v_list)[indices_v]
#h_test_v = np.array(h_test_v_list)[indices_v]
# Optional: Save the new test evaluation and validation sets to npy files (if needed)


Subject 70, Number of unique sessions: 19
Subject 86, Number of unique sessions: 19
Subject 96, Number of unique sessions: 18
Subject 125, Number of unique sessions: 19
Subject 129, Number of unique sessions: 5
Subject 167, Number of unique sessions: 2
Subject 174, Number of unique sessions: 19
Subject 175, Number of unique sessions: 6
Subject 183, Number of unique sessions: 6
Subject 194, Number of unique sessions: 19
Subject 197, Number of unique sessions: 19
Subject 199, Number of unique sessions: 19
Subject 201, Number of unique sessions: 19
Subject 203, Number of unique sessions: 6
Subject 206, Number of unique sessions: 6
Subject 207, Number of unique sessions: 41
Subject 214, Number of unique sessions: 18
Subject 219, Number of unique sessions: 5
Subject 230, Number of unique sessions: 6
Subject 236, Number of unique sessions: 42
Subject 245, Number of unique sessions: 6
Subject 250, Number of unique sessions: 41
Subject 257, Number of unique sessions: 6
Subject 260, Number of u

In [2]:
x_test_e.shape

(5000, 93, 129)

In [3]:
del x_test, x_test_e_list, x_test_v_list

In [4]:
with h5py.File("./data/neg_psd.h5", "r") as f:
    x_neg_r = f['data'][:]
    y_neg = f['labels'][:]
    s_neg = f['sessions'][:]


#X_neg_r_reshaped = X_neg_r.reshape(X_neg_r.shape[0], -1)
#X_neg_r_scaled = scaler.transform(X_neg_r_reshaped)
#X_neg_r = pca.transform(X_neg_r_scaled)

In [5]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Suppose x_test_e.shape is (N, C, L) = (100000, 93, 500)
# and x_test_v is for verification, shape (M, C, L).

############################
# Change 1: Define lists to store a separate scaler and PCA for each channel.
############################
scalers = []
pcas = []

############################
# Change 2: Create empty lists to collect the transformed PCA data 
# for x_test_e and x_test_v per channel.
############################
x_test_e_pca_list = []
x_test_v_pca_list = []
x_test_n_pca_list = []


x_test_e_pca_lists = []
x_test_v_pca_lists = []
x_test_n_pca_lists = []
############################
# Change 3: Iterate over each channel once, handle both x_test_e and x_test_v,
# and add progress output.
############################
num_channels = x_test_e.shape[1]

for ch in range(num_channels):
    # ---- Progress Report ----
    print(f"Processing channel {ch+1}/{num_channels}...")

    # Extract data for current channel from x_test_e and x_test_v
    channel_data_e = x_test_e[:, ch, :]  # shape (N, L)
    channel_data_v = x_test_v[:, ch, :]  # shape (M, L)
    channel_data_n = x_neg_r[:, ch, :]
    
    # 3A. Fit and transform x_test_e for this channel
    scaler_ch = StandardScaler()
    channel_data_e_scaled = scaler_ch.fit_transform(channel_data_e)
    channel_data_v_scaled = scaler_ch.transform(channel_data_v)
    channel_data_n_scaled =  scaler_ch.transform(channel_data_n)

    pca_ch = PCA(n_components=5)
    channel_data_e_pca = pca_ch.fit_transform(channel_data_e_scaled)
    channel_data_v_pca = pca_ch.transform(channel_data_v_scaled)
    channel_data_n_pca = pca_ch.transform(channel_data_n_scaled)
    
    # 3C. Save the scaler and PCA objects if needed later (for other data)
    #scalers.append(scaler_ch)
    #pcas.append(pca_ch)
    
    # 3D. Collect the PCA-transformed data for each channel
    x_test_e_pca_list.append(channel_data_e_pca)
    x_test_v_pca_list.append(channel_data_v_pca)
    x_test_n_pca_list.append(channel_data_n_pca)

############################
# Change 4: Stack the list back into single arrays.
# Now x_test_e has shape (N, C, 5) and x_test_v has shape (M, C, 5).
############################


x_test_e = np.stack(x_test_e_pca_list, axis=1)
x_test_v = np.stack(x_test_v_pca_list, axis=1)
x_test_n = np.stack(x_test_n_pca_list, axis=1)
############################
# Optional: Inspect explained variance ratio per channel if needed
############################
# for i, pca_ch in enumerate(pcas):
#     print(f"Channel {i}, PCA explained variance ratio: {pca_ch.explained_variance_ratio_}")

def compute_embedding_batch_two(x_test_batch, embedding_network=None):
    print(x_test_batch.shape)
    return x_test_batch


Processing channel 1/93...
Processing channel 2/93...
Processing channel 3/93...
Processing channel 4/93...
Processing channel 5/93...
Processing channel 6/93...
Processing channel 7/93...
Processing channel 8/93...
Processing channel 9/93...
Processing channel 10/93...
Processing channel 11/93...
Processing channel 12/93...
Processing channel 13/93...
Processing channel 14/93...
Processing channel 15/93...
Processing channel 16/93...
Processing channel 17/93...
Processing channel 18/93...
Processing channel 19/93...
Processing channel 20/93...
Processing channel 21/93...
Processing channel 22/93...
Processing channel 23/93...
Processing channel 24/93...
Processing channel 25/93...
Processing channel 26/93...
Processing channel 27/93...
Processing channel 28/93...
Processing channel 29/93...
Processing channel 30/93...
Processing channel 31/93...
Processing channel 32/93...
Processing channel 33/93...
Processing channel 34/93...
Processing channel 35/93...
Processing channel 36/93...
P

In [6]:
x_test_e.shape, x_test_v.shape

((5000, 93, 5), (80643, 93, 5))

In [7]:
channel_data_e.shape

(5000, 129)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from collections import defaultdict
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances as ed
from sklearn.metrics.pairwise import cosine_distances as cd
from collections import defaultdict
from pyeer.eer_info import get_eer_stats
from sklearn.metrics.pairwise import manhattan_distances as md
from sklearn.metrics.pairwise import cosine_similarity as cs


def EERf(resutls):
    resutls= np.array(resutls)
    genuine  = resutls[resutls[:, 1] == 1][:, 0]
    impostor = resutls[resutls[:, 1] == 0][:, 0]
    stats_a = get_eer_stats(genuine, impostor)
    return(stats_a.eer,stats_a.fmr100)

def calculate_and_print_averages(y_train, resutls3):
    u, counts = np.unique(y_train, return_counts=True)
    eer_values = []
    ii = 0

    for i in resutls3.keys():
        re = EERf(resutls3[i])
        eer = re[0]
        print(f"{i}: EER = {re[0]:.4f}, FMR100 = {re[1]:.4f}, Count = {counts[ii]}")
        eer_values.append(eer)
        ii += 1

    average_eer = np.mean(eer_values) * 100
    std_eer = np.std(eer_values) * 100
    
    print(f"Final Average EER: {average_eer:.4f}")
    print(f"Final EER Standard Deviation: {std_eer:.4f}")
    print(f"${average_eer:.2f} \\pm {std_eer:.2f}$")

import torch.nn.functional as F


def calculate_similarity_scores_two(enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance):
    similarity_results = []
    similarity_results_by_class = []
    similarity_results_by_class_dict = defaultdict(list)
    unique_classes = np.unique(y_enrollment)
    class_indices = [np.where(y_enrollment == cls)[0] for cls in unique_classes]
    print(verification_embeddings.shape, enrollment_embeddings.shape)
    if distance == "cd":
        similarity_matrix = -1 * cd(verification_embeddings, enrollment_embeddings)
    elif distance == "ed":
        similarity_matrix = -1 * ed(verification_embeddings, enrollment_embeddings)

    for i in range(similarity_matrix.shape[0]):
        current_class = y_verification[i]
        predicted_scores = similarity_matrix[i]
        same_class_indices = class_indices[np.where(unique_classes == current_class)[0][0]]

        for cls in unique_classes:
            same_class_indices = class_indices[np.where(unique_classes == cls)[0][0]]
            max_score = sum(sorted(predicted_scores[same_class_indices], reverse=True)[:10]) / 10
            if current_class == cls:
                similarity_results_by_class.append([max_score, 1, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 1, current_class, cls, i, cls])
            else:
                similarity_results_by_class.append([max_score, 0, current_class, cls, i])
                similarity_results_by_class_dict[cls].append([max_score, 0, current_class, cls, i, cls])

    return similarity_results_by_class, similarity_results_by_class_dict

def assessment_model_data_two(enrollment_data, ye, verification_data, yv, e_network, distance):
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    similarity_results_by_class, similarity_results_by_class_dict = calculate_similarity_scores_two(
        enrollment_embeddings, y_enrollment, verification_embeddings, y_verification,distance
    )
    return similarity_results_by_class, similarity_results_by_class_dict

def compute_embedding_batch_two(x_test_batch, embedding_network=None):
    print(x_test_batch.shape)
    return x_test_batch

In [9]:
#HEREAA

# Generate a sequence of indices from 0 to the length of one of the arrays



#, s_test_b
resutls2,resutls3=assessment_model_data_two(x_test_e.reshape(x_test_e.shape[0], -1), y_test_e, x_test_v.reshape(x_test_v.shape[0], -1), y_test_v, None, distance = "ed")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

(5000, 465)
(80643, 465)
(80643, 465) (5000, 465)
70: EER = 0.4818, FMR100 = 0.9367, Count = 100
86: EER = 0.3933, FMR100 = 0.7944, Count = 100
96: EER = 0.3076, FMR100 = 0.9747, Count = 100
125: EER = 0.3150, FMR100 = 0.7633, Count = 100


/home/fallahi/anaconda3/lib/python3.12/site-packages/pyeer/eer_stats.py:220: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


129: EER = 0.6025, FMR100 = 0.9750, Count = 100
167: EER = 0.6700, FMR100 = 1.0000, Count = 100
174: EER = 0.5467, FMR100 = 0.9411, Count = 100
175: EER = 0.8622, FMR100 = 0.9060, Count = 100
183: EER = 0.5940, FMR100 = 0.9620, Count = 100
194: EER = 0.6606, FMR100 = 0.8961, Count = 100
197: EER = 0.3844, FMR100 = 0.9117, Count = 100
199: EER = 0.4067, FMR100 = 0.9428, Count = 100
201: EER = 0.6140, FMR100 = 0.9150, Count = 100
203: EER = 0.3440, FMR100 = 0.4780, Count = 100
206: EER = 0.8305, FMR100 = 0.9360, Count = 100
207: EER = 0.5470, FMR100 = 0.9607, Count = 100
214: EER = 0.5475, FMR100 = 0.8976, Count = 100
219: EER = 0.4741, FMR100 = 0.6056, Count = 100
230: EER = 0.5080, FMR100 = 0.8100, Count = 100
236: EER = 0.6217, FMR100 = 0.9837, Count = 100
245: EER = 0.4380, FMR100 = 0.8620, Count = 100
250: EER = 0.5257, FMR100 = 0.9745, Count = 100
257: EER = 0.2260, FMR100 = 1.0000, Count = 100
260: EER = 0.4052, FMR100 = 0.9106, Count = 100
261: EER = 0.4400, FMR100 = 0.9733, Coun

In [10]:
#HEREAA

# Generate a sequence of indices from 0 to the length of one of the arrays



#, s_test_b
resutls2,resutls3=assessment_model_data_two(x_test_e.reshape(x_test_e.shape[0], -1), y_test_e, x_test_v.reshape(x_test_v.shape[0], -1), y_test_v, None, distance = "cd")
#print("simple",EERf(resutls))
calculate_and_print_averages(y_test_e, resutls3)

(5000, 465)
(80643, 465)
(80643, 465) (5000, 465)
70: EER = 0.4853, FMR100 = 0.9472, Count = 100
86: EER = 0.3928, FMR100 = 0.7950, Count = 100
96: EER = 0.3900, FMR100 = 0.9724, Count = 100
125: EER = 0.3157, FMR100 = 0.7272, Count = 100
129: EER = 0.6035, FMR100 = 0.9775, Count = 100
167: EER = 0.6700, FMR100 = 1.0000, Count = 100


/home/fallahi/anaconda3/lib/python3.12/site-packages/pyeer/eer_stats.py:220: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


174: EER = 0.5483, FMR100 = 0.9539, Count = 100
175: EER = 0.7720, FMR100 = 0.7800, Count = 100
183: EER = 0.5940, FMR100 = 0.9620, Count = 100
194: EER = 0.6736, FMR100 = 0.8978, Count = 100
197: EER = 0.3801, FMR100 = 0.9244, Count = 100
199: EER = 0.4006, FMR100 = 0.9461, Count = 100
201: EER = 0.6146, FMR100 = 0.9411, Count = 100
203: EER = 0.4100, FMR100 = 0.5520, Count = 100
206: EER = 0.7360, FMR100 = 0.8520, Count = 100
207: EER = 0.5490, FMR100 = 0.9633, Count = 100
214: EER = 0.5510, FMR100 = 0.8982, Count = 100
219: EER = 0.5388, FMR100 = 0.6207, Count = 100
230: EER = 0.5080, FMR100 = 0.8900, Count = 100
236: EER = 0.6225, FMR100 = 0.9866, Count = 100
245: EER = 0.4400, FMR100 = 0.8840, Count = 100
250: EER = 0.5247, FMR100 = 0.9802, Count = 100
257: EER = 0.4282, FMR100 = 0.9980, Count = 100
260: EER = 0.4067, FMR100 = 0.8933, Count = 100
261: EER = 0.4402, FMR100 = 0.9733, Count = 100
262: EER = 0.3628, FMR100 = 0.5920, Count = 100
265: EER = 0.2580, FMR100 = 0.9372, Coun

In [11]:
# Flatten each array from 3D to 2D
x_test_e = x_test_e.reshape(x_test_e.shape[0], -1)
x_test_v= x_test_v.reshape(x_test_v.shape[0], -1)
x_test_n = x_test_n.reshape(x_test_n.shape[0], -1)

In [15]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle


def classifiers_learn(enrollment_data, ye, verification_data, yv, e_network, negative_raw):
    """
    Learn classifiers per unique subject and evaluate using EER.
    """
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    
    # Compute embeddings
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    negative_embeddings = compute_embedding_batch_two(negative_raw, e_network)
    
    unique_classes = np.unique(y_enrollment)
    results = {}

    print("Unique classes: ", unique_classes)

    for cls in unique_classes:
        # Prepare training data
        positive_indices = np.where(y_enrollment == cls)[0]
        negative_indices = np.arange(len(negative_embeddings))
        
        train_embeddings = np.concatenate([enrollment_embeddings[positive_indices], negative_embeddings])
        train_labels = np.concatenate([np.ones(len(positive_indices)), np.zeros(len(negative_indices))])

        # Prepare testing data
        test_embeddings = verification_embeddings
        test_labels =(y_verification == cls).astype(int)


        # Initialize Incremental PCA
        #pca = IncrementalPCA(n_components=50)  # Retain 80% of variance or specify a fixed number of components
        

        #batch_size = 5000  # Define batch size
        #for i in range(0, train_embeddings.shape[0], batch_size):
            #batch = train_embeddings[i:i + batch_size]
            #pca.partial_fit(batch)  
        #train_embeddings_reduced = pca.transform(train_embeddings)
        #print("Reduced Training Shape:", train_embeddings_reduced.shape)
        #test_embeddings_reduced = pca.transform(test_embeddings)
        #print("Reduced Test Shape:", test_embeddings_reduced.shape)

        # Train Random Forest model
        #rf_model = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
        #rf_model.fit(train_embeddings, train_labels)
        #rf_probabilities = rf_model.predict_proba(test_embeddings)[:, 1]  # Use probabilities for EER calculation

        
        # Replace RandomForest with Linear SVM + Kernel Approximation
        pipeline = Pipeline([
            #('scaler', StandardScaler()),
            ('kernel_approximation', RBFSampler(gamma=1, random_state=42, n_components=500)),  # Approximate RBF kernel
            ('classifier', SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, class_weight='balanced', random_state=42, n_jobs=20))
        ])
        
        # Shuffle and split the data
        X, y = shuffle(train_embeddings, train_labels, random_state=42)
        
        # Train the pipeline
        pipeline.fit(X, y)
        
        # Predict probabilities for the test set
        # Note: SGDClassifier doesn't provide probabilities directly, so we use decision_function
        decision_scores = pipeline.decision_function(test_embeddings)
        probabilities = 1 / (1 + np.exp(-decision_scores))  # Sigmoid function to map decision scores to probabilities
        
        # Calculate EER
        svm_eer = calculate_eer(test_labels, probabilities)
        
        print(f"Class {cls}: Linear SVM with Kernel Approximation EER = {svm_eer:.4f}")
        
        results[cls] = {
            'RandomForest': {'EER': svm_eer}
        }
        
    return results



In [16]:
# Test your pipeline with synthetic data



resutls = classifiers_learn(
    x_test_e, y_test_e, x_test_v, y_test_v, None, x_test_n
)
# Print results
#print("EER:", EERf(resutls))

calculate_mean_std(resutls)

(5000, 465)
(80643, 465)
(84408, 465)
Unique classes:  [ 70  86  96 125 129 167 174 175 183 194 197 199 201 203 206 207 214 219
 230 236 245 250 257 260 261 262 265 268 270 281 291 296 299 303 306 314
 322 325 342 348 357 364 374 407 414 415 418 419 422 436]
genuine_scores:  1800 0.16522630124900953 impostor_scores 78843 0.10297767674449007
Class 70: Linear SVM with Kernel Approximation EER = 0.3661
genuine_scores:  1800 0.6902538130781166 impostor_scores 78843 0.5517351313891914
Class 86: Linear SVM with Kernel Approximation EER = 0.3528
genuine_scores:  1700 0.010009288082925203 impostor_scores 78943 0.011011972538703313
Class 96: Linear SVM with Kernel Approximation EER = 0.5171


/home/fallahi/anaconda3/lib/python3.12/site-packages/pyeer/eer_stats.py:220: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  1800 0.2764774238434562 impostor_scores 78843 0.11844494509959361
Class 125: Linear SVM with Kernel Approximation EER = 0.1844
genuine_scores:  400 0.2507747930680317 impostor_scores 80243 0.1612508231715818
Class 129: Linear SVM with Kernel Approximation EER = 0.4025
genuine_scores:  100 0.2436688140641785 impostor_scores 80543 0.12043063444684375
Class 167: Linear SVM with Kernel Approximation EER = 0.3339
genuine_scores:  1800 0.1962528959489236 impostor_scores 78843 0.15534976056853395
Class 174: Linear SVM with Kernel Approximation EER = 0.4407
genuine_scores:  500 0.32153946338428196 impostor_scores 80143 0.053563676487589515
Class 175: Linear SVM with Kernel Approximation EER = 0.1348
genuine_scores:  500 0.7423990907427852 impostor_scores 80143 0.3668967955486746
Class 183: Linear SVM with Kernel Approximation EER = 0.1900
genuine_scores:  1800 0.14442094575163636 impostor_scores 78843 0.09214396986009903
Class 194: Linear SVM with Kernel Approximation EER = 0.

/home/fallahi/anaconda3/lib/python3.12/site-packages/pyeer/eer_stats.py:220: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  4800 0.20791577862440533 impostor_scores 75843 0.15586234250284683
Class 260: Linear SVM with Kernel Approximation EER = 0.3531
genuine_scores:  900 0.17099192877293115 impostor_scores 79743 0.140088148081569
Class 261: Linear SVM with Kernel Approximation EER = 0.4290
genuine_scores:  500 0.5626999367075899 impostor_scores 80143 0.15875752560058984
Class 262: Linear SVM with Kernel Approximation EER = 0.0786
genuine_scores:  3900 0.05237151578526442 impostor_scores 76743 0.011336400512537781
Class 265: Linear SVM with Kernel Approximation EER = 0.4890
genuine_scores:  1800 0.7109454951050874 impostor_scores 78843 0.39056409120502833
Class 268: Linear SVM with Kernel Approximation EER = 0.2435
genuine_scores:  1800 0.03024019587134244 impostor_scores 78843 0.026240149189372907
Class 270: Linear SVM with Kernel Approximation EER = 0.3978
genuine_scores:  1800 0.42530899444561965 impostor_scores 78843 0.21688154845825303
Class 281: Linear SVM with Kernel Approximation EE

{'RandomForest': {'mean': 29.15748275119936, 'std': 14.034930683156327}}

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import IncrementalPCA
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle


def calculate_eer(y_true, y_scores):
    """
    Calculate the Equal Error Rate (EER) and FMR100 using pyeer.
    """
    # Separate genuine and impostor scores
    genuine_scores = y_scores[y_true == 1]
    impostor_scores = y_scores[y_true == 0]
    #print(y_true, y_scores)

    #print(genuine_scores, impostor_scores)
    print("genuine_scores: ", len(genuine_scores), sum(genuine_scores)/len(genuine_scores), "impostor_scores", len(impostor_scores), sum(impostor_scores)/len(impostor_scores))
    
    # Use pyeer to compute EER stats
    stats = get_eer_stats(genuine_scores, impostor_scores)
    
    return stats.eer

def classifiers_learn(enrollment_data, ye, verification_data, yv, e_network, negative_raw):
    """
    Learn classifiers per unique subject and evaluate using EER.
    """
    x_enrollment, y_enrollment = enrollment_data, ye
    x_verification, y_verification = verification_data, yv
    
    # Compute embeddings
    enrollment_embeddings = compute_embedding_batch_two(x_enrollment, e_network)
    verification_embeddings = compute_embedding_batch_two(x_verification, e_network)
    negative_embeddings = compute_embedding_batch_two(negative_raw, e_network)
    
    unique_classes = np.unique(y_enrollment)
    results = {}

    print("Unique classes: ", unique_classes)

    for cls in unique_classes:
        # Prepare training data
        positive_indices = np.where(y_enrollment == cls)[0]
        negative_indices = np.arange(len(negative_embeddings))
        
        train_embeddings = np.concatenate([enrollment_embeddings[positive_indices], negative_embeddings])
        train_labels = np.concatenate([np.ones(len(positive_indices)), np.zeros(len(negative_indices))])

        # Prepare testing data
        test_embeddings = verification_embeddings
        test_labels =(y_verification == cls).astype(int)


        # Initialize Incremental PCA
        #pca = IncrementalPCA(n_components=50)  # Retain 80% of variance or specify a fixed number of components
        

        #batch_size = 5000  # Define batch size
        #for i in range(0, train_embeddings.shape[0], batch_size):
            #batch = train_embeddings[i:i + batch_size]
            #pca.partial_fit(batch)  
        #train_embeddings_reduced = pca.transform(train_embeddings)
        #print("Reduced Training Shape:", train_embeddings_reduced.shape)
        #test_embeddings_reduced = pca.transform(test_embeddings)
        #print("Reduced Test Shape:", test_embeddings_reduced.shape)

        # Train Random Forest model
        #rf_model = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
        #rf_model.fit(train_embeddings, train_labels)
        #rf_probabilities = rf_model.predict_proba(test_embeddings)[:, 1]  # Use probabilities for EER calculation

        
        pipeline = Pipeline([
            #('scaler', StandardScaler()),
            ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=20))
            #('classifier', SVC(probability=True, random_state=42, class_weight='balanced'))
            #('classifier', KNeighborsClassifier(n_neighbors=20, weights='distance'))
        ])
        #svm_model = SVC(probability=True, random_state=42, class_weight='balanced')
        X, y = shuffle(train_embeddings, train_labels, random_state=42)
        pipeline.fit(X, y)
        probabilities = pipeline.predict_proba(test_embeddings)[:, 1]

        # Calculate EER
        rf_eer = calculate_eer(test_labels, probabilities)

        print(f"Class {cls}: RandomForest EER = {rf_eer:.4f}")

        results[cls] = {
            'RandomForest': {'EER': rf_eer}
        }

    return results

def calculate_mean_std(results):
    """
    Calculate the mean and standard deviation of EER values for each model in the results dictionary.
    
    Args:
        results (dict): A dictionary where keys are classes, and values are dictionaries containing model metrics.
                        Example: {cls: {'RandomForest': {'EER': value}}}
    
    Returns:
        dict: A dictionary containing mean and std for each model across all classes.
    """
    # Extract EER values for each model
    model_eer_values = {}
    for cls, metrics in results.items():
        for model, model_metrics in metrics.items():
            if model not in model_eer_values:
                model_eer_values[model] = []
            model_eer_values[model].append(model_metrics['EER'])
    
    # Calculate mean and std
    model_stats = {}
    for model, eer_values in model_eer_values.items():
        mean = np.mean(eer_values) * 100
        std = np.std(eer_values) * 100
        model_stats[model] = {'mean': mean, 'std': std}
        print(f"Final Average EER {model}: {mean:.4f}")
        print(f"Final EER Standard Deviation {model}: {std:.4f}")
        print(f"${mean:.2f} \\pm {std:.2f}$") 
        
   
    return model_stats


In [18]:
# Test your pipeline with synthetic data



resutls = classifiers_learn(
    x_test_e, y_test_e, x_test_v, y_test_v, None, x_test_n
)
# Print results
#print("EER:", EERf(resutls))

calculate_mean_std(resutls)

(5000, 465)
(80643, 465)
(84408, 465)
Unique classes:  [ 70  86  96 125 129 167 174 175 183 194 197 199 201 203 206 207 214 219
 230 236 245 250 257 260 261 262 265 268 270 281 291 296 299 303 306 314
 322 325 342 348 357 364 374 407 414 415 418 419 422 436]
genuine_scores:  1800 0.012955555555555583 impostor_scores 78843 0.0006736171885899784
Class 70: RandomForest EER = 0.3868
genuine_scores:  1800 0.053966666666666926 impostor_scores 78843 0.00041335311949064767
Class 86: RandomForest EER = 0.2346
genuine_scores:  1700 0.0025176470588235108 impostor_scores 78943 0.00010703925617217393
Class 96: RandomForest EER = 0.4014
genuine_scores:  1800 0.042411111111111136 impostor_scores 78843 0.00031340765825755673
Class 125: RandomForest EER = 0.3412
genuine_scores:  400 0.010374999999999968 impostor_scores 80243 0.0009238189000909845
Class 129: RandomForest EER = 0.3573
genuine_scores:  100 0.0007000000000000001 impostor_scores 80543 0.0006628757309759979
Class 167: RandomForest EER = 0.48

/home/fallahi/anaconda3/lib/python3.12/site-packages/pyeer/eer_stats.py:220: UserWarning: It is possible that you had set the wrong score type. Please consider reviewing if you are using dissimilarity or similarity scores
  warn("It is possible that you had set the wrong score"


genuine_scores:  4800 0.009529166666666674 impostor_scores 75843 0.0003188165025117739
Class 260: RandomForest EER = 0.3856
genuine_scores:  900 0.02077777777777777 impostor_scores 79743 0.0010555158446509593
Class 261: RandomForest EER = 0.4299
genuine_scores:  500 0.29684000000000027 impostor_scores 80143 0.0006165229651997043
Class 262: RandomForest EER = 0.0790
genuine_scores:  3900 0.021630769230769238 impostor_scores 76743 0.0006760225688336387
Class 265: RandomForest EER = 0.4644
genuine_scores:  1800 0.04057777777777775 impostor_scores 78843 0.0003178468602158782
Class 268: RandomForest EER = 0.3057
genuine_scores:  1800 0.03485555555555557 impostor_scores 78843 0.0005911748665068545
Class 270: RandomForest EER = 0.4797
genuine_scores:  1800 0.0028166666666666505 impostor_scores 78843 0.0002072473142828123
Class 281: RandomForest EER = 0.4331
genuine_scores:  1800 0.01232222222222223 impostor_scores 78843 0.0005250941744986893
Class 291: RandomForest EER = 0.4368
genuine_scores

{'RandomForest': {'mean': 32.027384391393525, 'std': 12.13123472355051}}

In [19]:
a = [ 70  86  96 125 129 167 174 175 183 194 197 199 201 203 206 207 214 219
 230 236 245 250 257 260 261 262 265 268 270 281 291 296 299 303 306 314
 322 325 342 348 357 364 374 407 414 415 418 419 422 436]

len(a)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1523724180.py, line 1)